In [1]:
import pandas as pd
import numpy as np
import ast

In [8]:
!pip install gradio
!pip install scikit-surprise


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 13.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3095415 sha256=60042f09092b8c78572f8c3061cb9fe56c5caef25640632b872700d57bf85430
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
dfmerge = pd.read_csv('/content/drive/MyDrive/merged_data6.csv')

dfmerge.head()

In [3]:
dfmerge = pd.read_csv('/content/drive/MyDrive/merged_data6.csv')

dfmerge.head()

,id,title,score_final,Drama,Comedy,Horror,Romance
0,862,Toy Story,1638.55,0,1,0,0
1,8844,Jumanji,735.17,0,0,0,0
2,15602,Grumpier Old Men,36.06,0,1,0,1
3,31357,Waiting to Exhale,14.57,1,1,0,1
4,11862,Father of the Bride Part II,58.37,0,1,0,0


In [9]:
import gradio as gr
import pandas as pd
import pickle
from surprise import SVD

def generar_recomendacion(svd_model, user_id, df, genres, top=5):
    # Filtrar las películas que correspondan a los géneros de interés
    df_filtered = df[df[genres].any(axis=1)]

    # Crear un mapeo de id de película a título de película para una búsqueda más eficiente
    id_to_title = df_filtered.set_index('id')['title'].to_dict()

    # Obtener las recomendaciones utilizando la función `apply` de pandas
    predicted_ratings = df_filtered['id'].apply(lambda movie_id: svd_model.predict(int(user_id), movie_id).est)

    # Ordenar las películas según su predicción de rating
    movie_rating = list(zip(df_filtered['id'], predicted_ratings))
    movie_rating.sort(key=lambda x: x[1], reverse=True)

    # Obtener los títulos de las películas recomendadas
    recommended_movies = movie_rating[:top]
    recommended_titles = [id_to_title[movie_id] for movie_id, _ in recommended_movies]

    # Devolver la lista de títulos como una cadena
    return ', '.join(recommended_titles)

# Leer los datos
dfmerge = pd.read_csv('/content/drive/MyDrive/merged_data6.csv')

# Cargar el modelo
with open('/content/drive/MyDrive/fc_model_svd_v1.pkl', 'rb') as file:
    svd_model = pickle.load(file)

def wrap_generar_recomendacion(user_id, drama, comedy, horror, romance, top=5):
    # Crear la lista de géneros de interés a partir de las casillas de verificación
    genres = []
    if drama: genres.append('Drama')
    if comedy: genres.append('Comedy')
    if horror: genres.append('Horror')
    if romance: genres.append('Romance')
    
    # Llamar a la función de recomendación y devolver los resultados como una cadena
    return generar_recomendacion(svd_model, user_id, dfmerge, genres, int(top))

# Definir la interfaz de Gradio
demo = gr.Interface(
    fn=wrap_generar_recomendacion,
    inputs=["text", gr.inputs.Checkbox(label="Drama"), gr.inputs.Checkbox(label="Comedy"), gr.inputs.Checkbox(label="Horror"), gr.inputs.Checkbox(label="Romance"), "text"], 
    outputs="text",
    title="Sistema de recomendación de películas basado en filtro colaborativo",
    description="Ingresa el ID del usuario (user_id), selecciona los géneros de interés y la cantidad de recomendaciones que te gustaría generar. Te mostraremos algunas películas que le pueden gustar.",
    allow_flagging='auto'
)

# Lanzar la interfaz
demo.launch()

/usr/local/lib/python3.10/dist-packages/gradio/inputs.py:121: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/inputs.py:124: UserWarning: `optional` parameter is deprecated, and it has no effect
  super().__init__(value=default, label=label, optional=optional)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>